In [4]:
import re
import mlflow
import pandas as pd
import matplotlib.pyplot as plt
import warnings

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [26]:
# Ignore warnings:
warnings.filterwarnings('ignore')

# Reading the dataset:
trainDF = pd.read_csv("../data/raw/titanic/train.csv")
testDF = pd.read_csv("../data/raw/titanic/test.csv")

def get_features(data):
    # Divide the data:
    target = "Survived"
    features = [col for col in data if col not in ['PassengerId','Survived', 'Name', 'Ticket', 'Cabin']] # Main features

    X = data[features]
    y = data[target]

    # Feature enginering:
    names = data['Name']
    pattern = re.compile(r",\s([A-Za-z]+)\.?") # Patter to get the title of the passengers
    titles = [match.group(1) for name in names if (match := pattern.search(name))] # Extract the title from names

    ticket_counts = data['Ticket'].value_counts() # Count the tickets to generate another feature

    # Adding the new features to the DF:
    X['Titles'] = titles
    X['TicketGroupSize'] = data['Ticket'].map(ticket_counts) # Map the ticket to the total count
    X['HasCabin'] = data['Cabin'].notnull().astype(int) # NaN means no cabin assigned = No first class passenger

    # Get numeric and categorical features:
    num_feat = [col for col in X if col not in ['Sex', 'Embarked', 'Titles', 'TicketGroupSize', 'HasCabin']]
    cat_feat = ['Sex', 'Embarked', 'Titles', 'TicketGroupSize', 'HasCabin']

    return X, y, num_feat, cat_feat

X_train, y_train, num_feat, cat_feat  = get_features(trainDF)

X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Titles,TicketGroupSize,HasCabin
0,3,male,22.0,1,0,7.2500,S,Mr,1,0
1,1,female,38.0,1,0,71.2833,C,Mrs,1,1
2,3,female,26.0,0,0,7.9250,S,Miss,1,0
3,1,female,35.0,1,0,53.1000,S,Mrs,2,1
4,3,male,35.0,0,0,8.0500,S,Mr,1,0


In [27]:
# Define the pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoding', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_feat),
    ('cat', categorical_transformer, cat_feat)
])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(random_state=42))
])